<h1 style="font-size: 70px;text-align: center; font-family:Copperplate; color: darkblue">Fake News Detection</h1>


<!-- # EDA -->
<div style = "height: 50px;
  width: 800px;
  background-color: darkblue;">
    <h1 style="padding: 10px;
              color:white;">
        <b>Introduction</b>
    </h1>

</div>

Naan Mudhalvan AI Fake News Detection Project.

<p style="font-size:30px; font-family:Papyrus"><b>Why should you care about whether or not your news is real or fake?</b></p>

<p style="font-family:Times New Roman ;font-size:20px">The spread of fake news has become easier in the digital age, as social media platforms and other online channels allow anyone to create and share content with the world. The problem with fake news is that it can be very convincing, and people may believe it to be true without questioning its authenticity. This can lead to a range of negative consequences, That is why a sophisticated method is required to identify fake news</p>

<h1 style="font-family:Papyrus">Notebook Overview:</h1>
<ol>
    <li><p style="font-family:Times New Roman ;font-size:20px">An extensive EDA of data present in different news articles</p></li>
    <li><p style="font-family:Times New Roman ;font-size:20px">The Goal of This notebook is to use Machine Learning to Detect Fake News</p></li>
</ol>

In [ ]:
import os

BASE_DIR = os.getcwd()
DATA_PATH = os.path.join(
    BASE_DIR,
    "data",
    "WELFake_Dataset.csv"
)


<!-- # EDA -->
<div id = "eda" style = "height: 50px;
  width: 800px;
  background-color: #813EEC;">
    <h1 style="padding: 10px;
              color:white;">
        <b>1.EDA</b>
    </h1>

</div>

In [ ]:
#importing Libraries
import numpy as np
import pandas as pd
from matplotlib.pylab import plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

<div id = "rd"style = "height: 50px;
  width: 800px;
  background-color: #00f349;">
    <h3 style="padding: 10px;
              color:black;">
       1.1 Reading Data
    </h3>

</div>

In [ ]:
df = pd.read_csv(DATA_PATH)
df.head(10)

In [ ]:
df['title'] = df['title'].replace('', np.nan)
df['text'] = df['text'].replace('', np.nan)
df.dropna(inplace=True)
df.head()

In [ ]:
df.iloc[:1000, :3].to_csv("game_data1.csv", index=False)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
y = df.label
print(f'Ratio of real and fake news:')
y.value_counts(normalize=True).rename({1: 'real', 0: 'fake'})

In [ ]:
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

In [ ]:
df.isnull().sum().plot(kind="barh")
plt.show()

<div style = "height: 210px;
  width: 800px;
  border:5px inset red;
  background-color: #e2e1e9;">
    <h3 style="
              color:black;
               ">
        <b>Observations:</b>
        
 <p>
     <ol>
         <li>There are a total of 4 columns and 72134 rows in the data</li>
         <li>Label is the target variable</li>
         <li>Percentage of Real and fake News articles:<ul><li>real : 51%</li><li>fake :49%</li></ul></li>
         <li>Missing values are present in the dataset</li>
    </ol>
 </p>
    </h3>

</div>

In [ ]:
df.isnull().sum()

In [ ]:
df = df.fillna('')

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
df["title_text"] = df["title"] + df["text"]
df["body_len"] = df["title_text"].apply(lambda x: len(x) - x.count(" "))
df.head()

In [ ]:
bins = np.linspace(0, 200, 40)

plt.hist(df[df["label"]== 1]["body_len"], bins, alpha=0.5, label="Fake", color="#FF5733")
plt.hist(df[df["label"]== 0]["body_len"], bins, alpha=0.5, label="Real", color="#33FFB8")
plt.legend(loc="upper left")
plt.show()

<div style = "height: 150px;
  width: 800px;
  border:5px inset red;
  background-color: #e2e1e9;">
    <h3 style="padding: 10px;
              color:black;
               ">
        <b>Insights :</b>
        <br></br>
       
 <i>Fake news seem to be quite a bit longer than real news.</i>
    </h3>

</div>

In [ ]:
class_names = ['fake', 'real']
label_count = df.label.value_counts()
sns.barplot(x=label_count.index, y=label_count)
plt.title('Distribution of Fake/Real News',fontsize =14)

In [ ]:
y.head()

In [ ]:
df.tail()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['title_text'], y, test_size=0.33, random_state=53)

<div id = "nt"style = "height: 50px;
  width: 800px;
  background-color: #00f349;">
    <h3 style="padding: 10px;
              color:black;">
       1.2 Data Visualization of all News Titles
    </h3>

</div>

In [ ]:
titles = ' '.join(title for title in df['title'])

wordcloud = WordCloud(
    background_color='white',
    max_words=300,
    width=800,
    height=400
).generate(titles)

plt.figure(figsize=(20, 10))
plt.imshow(wordcloud.to_image(), interpolation='bilinear')
plt.axis('off')
plt.show()


<div style = "height: 110px;
  width: 800px;
  border:5px inset red;
  background-color: #e2e1e9;">
    <h3 style="
              color:black;
               ">
        <b>Quick Insights:</b>
        
 <p>
     <ul>
         <li>many of available news articles are about elections and presidents of USA </li>
    </ul>
 </p>
    </h3>

</div>

<div id = "ft"style = "height: 50px;
  width: 800px;
  background-color: #00f349;">
    <h3 style="padding: 10px;
              color:black;">
       1.3 Data Visualization of texts in fake news
    </h3>

</div>

In [ ]:
fake_news = X_train[y_train == 0]
real_news = X_train[y_train == 1]
fake_texts = ' '.join(text for text in fake_news)
wordcloud = WordCloud(
    background_color='white',
    max_words=300,
    width=800,
    height=400,
).generate(fake_texts)

plt.figure(figsize=(20, 10))
plt.imshow(wordcloud.to_image(), interpolation='bilinear')
plt.axis("off")
plt.show()

<div id = "rt"style = "height: 50px;
  width: 800px;
  background-color: #00f349;">
    <h3 style="padding: 10px;
              color:black;">
       1.4 Data Visualization of texts in Real news
    </h3>

</div>

In [ ]:
real_texts = ' '.join(text for text in real_news)
wordcloud = WordCloud(
    background_color='white',
    max_words=300,
    width=800,
    height=400,
).generate(real_texts)

plt.figure(figsize=(20, 10))
plt.imshow(wordcloud.to_image(), interpolation='bilinear')
plt.axis("off")
plt.show

<!-- # EDA -->
<div id = "bm"style = "height: 50px;
  width: 800px;
  background-color: #813EEC;">
    <h1 style="padding: 10px;
              color:white;">
        <b>2.Bag of Words Model</b>
    </h1>

</div>

In [ ]:
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)


In [ ]:
import pickle
pickle.dump(count_vectorizer, open("vectorizer.pkl", "wb"))

<!-- # EDA -->
<div id = "nb"style = "height: 50px;
  width: 800px;
  background-color: #813EEC;">
    <h1 style="padding: 10px;
              color:white;">
        <b>3.Training Naive Bayes Model</b>
    </h1>

</div>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB()
nb_classifier.fit(count_train, y_train)


<div id = "p1"style = "height: 50px;
  width: 800px;
  background-color: #00f349;">
    <h3 style="padding: 10px;
              color:black;">
       3.1 Predictions on NB model
    </h3>

</div>

In [ ]:
count_test = count_vectorizer.transform(X_test)
pred = nb_classifier.predict(count_test)

In [ ]:
pickle.dump(nb_classifier, open("nb_model.pkl", "wb"))

In [ ]:
my_model = pickle.load(open("nb_model.pkl", "rb"))

In [ ]:
count_vectorizer2 = CountVectorizer(stop_words='english')
count_train = count_vectorizer2.fit_transform(X_train)


In [ ]:
pickle.dump(count_vectorizer2, open("vectorizer.pkl", "wb"))

<div id = "e1"style = "height: 50px;
  width: 800px;
  background-color: #00f349;">
    <h3 style="padding: 10px;
              color:black;">
       3.2 Evaluation of NB predictions
    </h3>

</div>

In [ ]:
print(classification_report(y_test, pred))

<div style = "height: 65px;
  width: 800px;
  border:5px inset red;
  background-color: #e2e1e9;">
    <h3 style="
              color:black;
               text-align: center;
               ">
        <i><b>Accuracy achieved using Naive Bayes Model : 90%</b></i>
    </h3>

</div>

<div id = "cm1"style = "height: 50px;
  width: 800px;
  background-color: #00f349;">
    <h3 style="padding: 10px;
              color:black;">
       3.3 Confusion Matrix
    </h3>

</div>

In [ ]:
plt.figure(figsize = (8,6))

sns.heatmap(confusion_matrix(y_test,pred), annot=True,
            fmt='', cmap='Blues')

plt.xlabel('Predicted Labels')
plt.ylabel('Real Labels')